# Facial Recognition Software

In [1]:
#1. Generate dataset 
#2. Train the classifier and save it 
#3. Detect the face and name it if its already stored in our dataset

# Generate dataset

In [2]:
import cv2
import os 
import numpy as np
from PIL import Image

In [3]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y + h, x:x + w]
        return cropped_face

    cap = cv2.VideoCapture(0)
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            directory = "data"
            if not os.path.exists(directory):  # Check if directory exists
                os.makedirs(directory)  # Create directory if it doesn't exist
            file_name_path = f"{directory}/user_{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 13 or img_id == 50:
                break
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed..............")

generate_dataset()

# Train the classifier and save it

In [4]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert image to grayscale
        image_np = np.array(img, 'uint8')
        
        # Split filename and handle errors
        try:
            id = int(os.path.split(image)[1].split('_')[1].split('.')[0])  # Extract ID from filename
        except IndexError:
            print("Error processing filename:", image)
            continue

        faces.append(image_np)
        ids.append(id)

    # Define and initialize the classifier
    clf = cv2.face.createLBPHFaceRecognizer()

    # Train the classifier with faces and IDs
    clf.train(faces, np.array(ids))

    # Write the trained model to a file
    clf.save("classifier.xml")  # Save method instead of write

train_classifier("data")

AttributeError: module 'cv2' has no attribute 'face'